In [1]:
import cv2 as cv
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import Video
import ipywidgets as widgets

import detector.detect as detect

import time

%matplotlib inline

images_root = Path.cwd()/"test-images"
video_source = images_root/"ring-video-h264.mp4"

In [2]:
def convert_to_h264(path):
    if not path.exists():
        return
    converted_path = path.as_posix() + ".h264.mp4"
    !ffmpeg -y -i {path.as_posix()} -c:v libx264 {converted_path} -hide_banner -loglevel error
    !mv {converted_path} {path.as_posix()}

In [3]:
target_path = Path.cwd()/"ring-targeting.mp4"
lower = np.array([7, 85, 149])
upper = np.array([32, 216, 255])

scaling_factor = 0.3

video = cv.VideoCapture(video_source.as_posix())
width = int(video.get(cv.CAP_PROP_FRAME_WIDTH) * scaling_factor)
height = int(video.get(cv.CAP_PROP_FRAME_HEIGHT) * scaling_factor)
dims = (width, height)
fps = video.get(cv.CAP_PROP_FPS)/1

targeting_video = cv.VideoWriter(target_path.as_posix(), cv.VideoWriter_fourcc(*"mp4v"), fps, dims)
start = time.time()
now = time.time()
steps = []
match_count = []
while video.isOpened():
    ok, frame = video.read()
    now = time.time()
    if not ok:
        break

    frame = cv.resize(frame, dims)
    
    hsl = cv.cvtColor(frame, cv.COLOR_BGR2HLS_FULL)
    matches = detect.ring(hsl, lower, upper)
    #out = frame
    out = cv.cvtColor(hsl, cv.COLOR_HLS2BGR_FULL)
    for m in matches:
        out = m.show(out, line_weight=2)

    steps.append(time.time() - now)
    match_count.append(len(matches))
    targeting_video.write(out)

duration = time.time() - start
per_frame = duration / video.get(cv.CAP_PROP_FRAME_COUNT)
print(f"processing time: {duration:.3f}s ({1/per_frame:.1f}fps) @ {dims[0]}x{dims[1]})")
print(f"avg: {np.mean(steps)*1000:.2f}ms ({np.mean(match_count):.2f} matches)")
min_frame = min(range(len(steps)), key=steps.__getitem__)
max_frame = max(range(len(steps)), key=steps.__getitem__)
print(f"min: {min(steps)*1000:.2f}ms (frame {min_frame}: {match_count[min_frame]} matches)")
print(f"max: {max(steps)*1000:.2f}ms (frame {max_frame}: {match_count[max_frame]} matches)")

video.release()
targeting_video.release()

convert_to_h264(target_path)
Video(target_path.as_posix(), embed=True)

processing time: 0.723s (165.9fps) @ 576x324)
avg: 2.20ms (1.94 matches)
min: 1.68ms (frame 41: 1 matches)
max: 5.78ms (frame 0: 2 matches)
